In [1]:
import numpy as np
from datetime import datetime

In [2]:
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [3]:
class LogicGate:
    
    def __init__(self, gate_name, i_node, h1_node, h2_node, o_node, learning_rate):  
        self.name = gate_name
        
        self.W2 = np.random.rand(i_node, h1_node)             
        self.b2 = np.random.rand(h1_node)
        
        self.W3 = np.random.rand(h1_node, h2_node)
        self.b3 = np.random.rand(h2_node)
        
        self.W4 = np.random.rand(h2_node, o_node) 
        self.b4 = np.random.rand(o_node)
                        
        self.learning_rate = learning_rate
        
        print(self.name + " object is created")
        print('\n=================================================')
        print('initial W2 =', self.W2)
        print('initial b2 =', self.b2)
        print('initial W3 =', self.W3)
        print('initial b3 =', self.b3)
        print('=================================================\n')

    def feed_forward(self):
        delta = 1e-7   
        
        Z2 = np.dot(self.xdata, self.W2) + self.b2
        A2 = sigmoid(Z2)
        
        Z3 = np.dot(A2, self.W3) + self.b3
        A3 = sigmoid(Z3)
        
        Z4 = np.dot(A3, self.W4) + self.b4
        y = sigmoid(Z4)
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    def loss_val(self):
        delta = 1e-7   
    
        Z2 = np.dot(self.xdata, self.W2) + self.b2
        A2 = sigmoid(Z2)
        
        Z3 = np.dot(A2, self.W3) + self.b3
        A3 = sigmoid(Z3)
        
        Z4 = np.dot(A3, self.W4) + self.b4
        y = sigmoid(Z4)   
        
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
                       
    def train(self, xdata, tdata):
        self.xdata = xdata
        self.tdata = tdata
        
        f = lambda x : self.feed_forward()
      
        self.W2 -= self.learning_rate * numerical_derivative( f, self.W2 )
        self.b2 -= self.learning_rate * numerical_derivative( f, self.b2 )

        self.W3 -= self.learning_rate * numerical_derivative( f, self.W3 )
        self.b3 -= self.learning_rate * numerical_derivative( f, self.b3 )
        
        self.W4 -= self.learning_rate * numerical_derivative( f, self.W4 )
        self.b4 -= self.learning_rate * numerical_derivative( f, self.b4 )
                       
    def predict(self, x):
        Z2 = np.dot(x, self.W2) + self.b2
        A2 = sigmoid(Z2)
        
        Z3 = np.dot(A2, self.W3) + self.b3
        A3 = sigmoid(Z3)
        
        Z4 = np.dot(A3, self.W4) + self.b4
        y = sigmoid(Z4)  
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy1(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list
    
    def accuracy2(self, test_data):
        matched_list = []
        not_matched_list = []
        
        test_xdata = test_data[:, 0:-1]
        test_tdata = test_data[:, -1]
        
        for index in range(len(test_xdata)):            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
        
        accuracy_result = len(matched_list) / len(test_xdata)
        print("\nAccuracy =>", accuracy_result)
        
        return matched_list, not_matched_list

## AND

In [12]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([0, 0, 0, 1]).reshape(4, 1)
i_node = 2
h1_node = 1
h2_node = 1
o_node = 1
learning_rate = 1e-1
iteration_count = 10001

AND_obj = LogicGate("AND_GATE", i_node, h1_node, h2_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for step in  range(iteration_count):
    AND_obj.train(xdata, tdata)
    
    if (step % 1000 == 0):
        print("step = ", step, "loss value =", AND_obj.loss_val())
                
end_time = datetime.now()

print("\nElapsed Time =>", end_time - start_time)

AND_GATE object is created

initial W2 = [[0.54593402]
 [0.42825308]]
initial b2 = [0.51335713]
initial W3 = [[0.92783171]]
initial b3 = [0.71550506]

step =  0 loss value = 3.0260275894880224
step =  1000 loss value = 2.2364462798581
step =  2000 loss value = 0.1440071289389989
step =  3000 loss value = 0.03736604479149418
step =  4000 loss value = 0.021313667346394942
step =  5000 loss value = 0.014899361738364134
step =  6000 loss value = 0.011451577053529467
step =  7000 loss value = 0.00929961223996789
step =  8000 loss value = 0.007828665841437146
step =  9000 loss value = 0.006759665634358111
step =  10000 loss value = 0.005947670287525246

Elapsed Time => 0:00:04.945378


In [13]:
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = AND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = AND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = AND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

AND_GATE 

real_val = [0.00093914] , logical_val = 0
real_val = [0.00129551] , logical_val = 0
real_val = [0.00129551] , logical_val = 0
real_val = [0.99758713] , logical_val = 1

Accuracy => 1.0

 ([0, 1, 2, 3], [])

Accuracy => 1.0

 ([0, 1, 2, 3], [])


## OR

In [14]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([0, 1, 1, 1]).reshape(4, 1)
i_node = 2
h1_node = 1
h2_node = 1
o_node = 1
learning_rate = 1e-1
iteration_count = 10001

OR_obj = LogicGate("OR_GATE", i_node, h1_node, h2_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for step in  range(iteration_count):
    OR_obj.train(xdata, tdata)
    
    if (step % 1000 == 0):
        print("step =", step, "loss value =", OR_obj.loss_val())
                
end_time = datetime.now()

print("\nElapsed Time =>", end_time - start_time)

OR_GATE object is created

initial W2 = [[0.41508414]
 [0.99168702]]
initial b2 = [0.99072733]
initial W3 = [[0.11013595]]
initial b3 = [0.73437581]

step = 0 loss value = 2.2495369356034667
step = 1000 loss value = 1.0131312428152917
step = 2000 loss value = 0.05331227467634482
step = 3000 loss value = 0.025988932587789094
step = 4000 loss value = 0.017134319744051447
step = 5000 loss value = 0.012770231420892897
step = 6000 loss value = 0.010174763861753932
step = 7000 loss value = 0.008454816340502618
step = 8000 loss value = 0.007231649567799246
step = 9000 loss value = 0.006317336341913988
step = 10000 loss value = 0.005608080177509813

Elapsed Time => 0:00:04.689522


In [15]:
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

for data in test_xdata:
    (real_val, logical_val) = OR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = OR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = OR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

OR_GATE 

real_val = [0.00333221] , logical_val = 0
real_val = [0.9991887] , logical_val = 1
real_val = [0.99918796] , logical_val = 1
real_val = [0.99935351] , logical_val = 1

Accuracy => 1.0

 ([0, 1, 2, 3], [])

Accuracy => 1.0

 ([0, 1, 2, 3], [])


## NAND

In [16]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([1, 1, 1, 0]).reshape(4, 1)
i_node = 2
h1_node = 1
h2_node = 1
o_node = 1
learning_rate = 1e-1
iteration_count = 10001

NAND_obj = LogicGate("NAND_GATE", i_node, h1_node, h2_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for step in  range(iteration_count):
    NAND_obj.train(xdata, tdata)
    
    if (step % 1000 == 0):
        print("step =", step, "loss value =", NAND_obj.loss_val())
                
end_time = datetime.now()

print("\nElapsed Time =>", end_time - start_time)

NAND_GATE object is created

initial W2 = [[0.09666921]
 [0.34166611]]
initial b2 = [0.4525668]
initial W3 = [[0.16714825]]
initial b3 = [0.02551794]

step = 0 loss value = 2.2867842668014275
step = 1000 loss value = 2.1817127944751795
step = 2000 loss value = 0.08019861786790933
step = 3000 loss value = 0.0310588054551462
step = 4000 loss value = 0.019159820516275492
step = 5000 loss value = 0.013839396727693788
step = 6000 loss value = 0.010828289953391359
step = 7000 loss value = 0.008892230373441029
step = 8000 loss value = 0.007543060100927791
step = 9000 loss value = 0.006549182425413056
step = 10000 loss value = 0.005786642519339761

Elapsed Time => 0:00:04.620596


In [17]:
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = NAND_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = NAND_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = NAND_obj.accuracy2(test_data)

print("\n", accuracy_ret)

NAND_GATE 

real_val = [0.99943682] , logical_val = 1
real_val = [0.99911327] , logical_val = 1
real_val = [0.99911327] , logical_val = 1
real_val = [0.00344351] , logical_val = 0

Accuracy => 1.0

 ([0, 1, 2, 3], [])

Accuracy => 1.0

 ([0, 1, 2, 3], [])


## XOR

In [18]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ]).reshape(4, 2)
tdata = np.array([0, 1, 1, 0]).reshape(4, 1)
i_node = 2
h1_node = 2
h2_node = 2
o_node = 1
learning_rate = 1e-1
iteration_count = 10001

XOR_obj = LogicGate("XOR_GATE", i_node, h1_node, h2_node, o_node, learning_rate)
        
start_time = datetime.now()  
        
for step in  range(iteration_count):
    XOR_obj.train(xdata, tdata)
    
    if (step % 1000 == 0):
        print("step =", step, "loss value =", XOR_obj.loss_val())
                
end_time = datetime.now()

print("\nElapsed Time =>", end_time - start_time)

XOR_GATE object is created

initial W2 = [[0.28460116 0.16250596]
 [0.30003582 0.90433126]]
initial b2 = [0.98706257 0.95528488]
initial W3 = [[0.70078005 0.83650956]
 [0.91653672 0.69217958]]
initial b3 = [0.04124792 0.46657305]

step = 0 loss value = 3.6486086488676355
step = 1000 loss value = 2.7721476651460937
step = 2000 loss value = 2.7707971027948446
step = 3000 loss value = 2.762657385184312
step = 4000 loss value = 2.367852486993404
step = 5000 loss value = 1.9609376289526779
step = 6000 loss value = 0.35281352905966035
step = 7000 loss value = 0.03055049686833705
step = 8000 loss value = 0.01564947513193496
step = 9000 loss value = 0.010485717934710866
step = 10000 loss value = 0.007873244901413454

Elapsed Time => 0:00:09.418505


In [19]:
print(XOR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

for data in test_xdata:
    (real_val, logical_val) = XOR_obj.predict(data)
    print("real_val =", real_val, ", logical_val =", logical_val)

accuracy_ret = XOR_obj.accuracy1(test_xdata, test_tdata)

print("\n", accuracy_ret)

accuracy_ret = XOR_obj.accuracy2(test_data)

print("\n", accuracy_ret)

XOR_GATE 

real_val = [0.00234123] , logical_val = 0
real_val = [0.99854216] , logical_val = 1
real_val = [0.99854217] , logical_val = 1
real_val = [0.00260847] , logical_val = 0

Accuracy => 1.0

 ([0, 1, 2, 3], [])

Accuracy => 1.0

 ([0, 1, 2, 3], [])
